# Setup

In [30]:
import pandas as pd
import numpy as np
import random

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [5]:
data = {"AAGGA" : 0,
        "AAGGA" : 1,
        "AAGGA" : 2, 
        "AAGGT" : 3, 
        "ATGGA" : 4, }
        # . . . 66

In [7]:
input=torch.tensor([0, 1, 2])
input

tensor([0, 1, 2])

In [9]:
embeddings = nn.Embedding(66, 2)
embeddings(input)

tensor([[-0.0235, -1.2632],
        [ 0.4791,  0.8433],
        [ 2.8003,  2.0574]], grad_fn=<EmbeddingBackward0>)

# Linear Regression Trial

In [10]:
import torch
# Import the neural network module from pytorch
import torch.nn as nn

# Linear regression
# f = w * x 
# here : f = 2 * x

# 0) Training samples, watch the shape!. Here we have (8, 1) for 8 observations of 1 feature each
X = torch.tensor([[1], [2], [3], [4], [5], [6], [7], [8]], dtype=torch.float32)
Y = torch.tensor([[2], [4], [6], [8], [10], [12], [14], [16]], dtype=torch.float32)

n_samples, n_features = X.shape
print(f'n_samples = {n_samples}, n_features = {n_features}')

# 0) create a test sample
X_test = torch.tensor([5], dtype=torch.float32)

n_samples = 8, n_features = 1


In [11]:
# 1) Design Model, the model has to implement the forward pass!

# Here we could simply use a built-in model from PyTorch
# model = nn.Linear(input_size, output_size)

# Pytorch model class must ALWYAS inherit from nn.module
class LinearRegression(nn.Module):
    # Must always init the pytorch model class
    def __init__(self, input_dim, output_dim):
        # Need to super init
        super(LinearRegression, self).__init__()
        # define different layers. Here there is only one linear for the linear regression
        # nn.Linear performs the linear regression operation w*x + b
        self.lin = nn.Linear(input_dim, output_dim)

    # Apply the layers. Need to include x in function signature so model has input
    def forward(self, x):
        return self.lin(x)

# Specifying the input and output dimensions
input_size, output_size = n_features, n_features
# Insantiate Linear Regression Neural Network Model
model = LinearRegression(input_size, output_size)

print(f'Prediction before training: f({X_test.item()}) = {model(X_test).item():.3f}')

# 2) Define loss and optimizer
learning_rate = 0.01
n_epochs = 100
# Automatically implements the MSE formula
loss = nn.MSELoss()
# Use Stochastic Gradient Descent. Need to supply the model parameters 
# and a selected learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# 3) Training loop
for epoch in range(n_epochs):
    # predict = forward pass with our model
    # Internally this calls model.forward(x), performing the linear regression
    # and returning the predicted y_values
    y_predicted = model(X)

    # loss
    l = loss(Y, y_predicted)

    # calculate gradients = backward pass
    l.backward()

    # update weights. This updates model.parameters()
    optimizer.step()

    # zero the gradients after updating
    optimizer.zero_grad()

    if (epoch+1) % 10 == 0:
        w, b = model.parameters() # unpack parameters. In this instance the weight & the bias
        print('epoch ', epoch+1, ': w = ', w[0][0].item(), ' loss = ', l.item())
        

print(f'Prediction after training: f({X_test.item()}) = {model(X_test).item():.3f}')

Prediction before training: f(5.0) = -2.208
epoch  10 : w =  1.9433797597885132  loss =  0.020263902842998505
epoch  20 : w =  1.9468997716903687  loss =  0.018501583486795425
epoch  30 : w =  1.9489827156066895  loss =  0.017078982666134834
epoch  40 : w =  1.9509832859039307  loss =  0.015765808522701263
epoch  50 : w =  1.9529054164886475  loss =  0.014553562738001347
epoch  60 : w =  1.9547522068023682  loss =  0.013434533029794693
epoch  70 : w =  1.956526517868042  loss =  0.012401577085256577
epoch  80 : w =  1.9582313299179077  loss =  0.011448007076978683
epoch  90 : w =  1.959869146347046  loss =  0.010567796416580677
epoch  100 : w =  1.9614428281784058  loss =  0.009755214676260948
Prediction after training: f(5.0) = 10.024


# Trial

## Data Loaders

In [1]:
# To be able to import the file
import sys
pathname="/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts"
if pathname not in sys.path:
    sys.path.append(pathname)
path_to_data = "/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/data/data.json"
path_to_labels = "/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/data/data.info"

In [24]:
import prepData as prepD
import importlib
importlib.reload(prepD)

<module 'prepData' from '/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts/prepData.py'>

In [3]:
import getData as gd
getData = gd.getData(path_to_data=path_to_data, path_to_labels=path_to_labels)
label_df = getData.get_labels()

In [4]:
# With train - test split
train_genes = prepD.splitdata(label_df = label_df)

In [15]:
trainData = prepD.prepData(train_genes=train_genes, train_set=True, 
                          path_to_data=path_to_data, path_to_labels=path_to_labels, 
                          num_entries=10000)

In [20]:
train_dataLoader = trainData.get_data_loader(batchsize=256)

In [25]:
testData = prepD.prepData(train_genes=train_genes, train_set=False, 
                          path_to_data=path_to_data, path_to_labels=path_to_labels, 
                         num_entries=10000)

In [26]:
test_dataLoader = testData.get_data_loader(batchsize=256, oversample=False, shuffle=False)

# Build Neural Network

In [36]:
import torch
import torch.nn as nn

In [40]:
class m6aNet(nn.Module):
    def __init__(self, batchsize, readsize):
        self.batchsize = batchsize
        self.readsize = readsize
        super(m6aNet, self).__init__()
        # Embedding Layer
        self.embed = nn.Embedding(66, 2)

        ## First Layer ##
        self.read_level_prob_1 = nn.Linear(15, 150)
        # First Batch Norm Layer
        self.norm_1 = nn.BatchNorm1d(num_features=150)
        # First Activation Layer
        self.activ_1=nn.ReLU()
        # First Dropout Layer
        self.drop_1 = nn.Dropout(p=0.00)

        ## Second Layer ##
        self.read_level_prob_2 = nn.Linear(150, 32)
        # Second Activation Layer
        self.activ_2=nn.ReLU()
        # Second Dropout Layer
        self.drop_2 = nn.Dropout(p=0.00)

        ## Third Layer ##
        self.read_level_prob_3 = nn.Linear(32, 1)
        # Sigmoid Activation
        self.sig_1 = nn.Sigmoid()

    
    
    def forward(self, x):
        ### X is a tensor of shape (batchsize, readsize=20, 12) ###
        
        # Extract numeric features        
        numerics = x[:, :, :9]
        # # Extract Bases
        bases = x[:, :, 9:].type(torch.int64)
        # # Feed to embedding layer
        bases = self.embed(bases)

        # Reshape
        bases = bases.reshape(-1, self.readsize, 3*2)
        # Combine embedded output with numeric features
        x = torch.concat((numerics, bases), 2).type(torch.float)

        #### Feed Forward  ####

        ## First Layer ##
        x = self.read_level_prob_1(x)
        # First Batch Norm Layer
        x = x.transpose(dim0=1, dim1=2) # Need to transpose first
        x = self.norm_1(x)
        x = x.transpose(dim0=1, dim1=2) # Then transpose back
        # First Activation Layer
        x= self.activ_1(x)
        # First Dropout Layer
        x = self.drop_1(x)

        ## Second Layer ##
        x = self.read_level_prob_2(x)
        # Second Activation Layer
        x = self.activ_2(x)
        # Second Dropout Layer
        x = self.drop_2(x)

        ## Third Layer ##
        x = self.read_level_prob_3(x)
        # Sigmoid Activation
        x = self.sig_1(x)
        x = x.reshape(-1, self.readsize)
        
        # Final Output
        r = 1 - torch.prod(1 - x, axis=1)
        return r

            
        
        
        
        
        

In [29]:
model=m6aNet(batchsize=50, readsize=20)
results = model.forward(X)

## Training Parameters

In [51]:
batchsize = 256

# Instantiate model
model = m6aNet(batchsize=batchsize, readsize=20)

# Training Parameters
learning_rate = 0.01
num_epochs = 10
n_steps = len(train_dataLoader)


# Automatically implements the MSE formula
criterion = nn.BCELoss()
# Use Stochastic Gradient Descent. Need to supply the model parameters 
# and a selected learning rate
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0)

## Training Loop

In [46]:
0.7*121838

85286.59999999999

In [47]:
85286/256

333.1484375

In [48]:
n_steps

39

In [52]:
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_dataLoader):
        # Flatten labels
        labels = labels.flatten().float()
        
        # Forward pass and loss calculation
        outputs = model(features)
        loss = criterion(outputs, labels)
        
        # Backward & Optimize
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if (i+1) % 10 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_steps}], Loss: {loss.item():.4f}')

Epoch [1/10], Step [10/39], Loss: 1.1977
Epoch [1/10], Step [20/39], Loss: 0.5566
Epoch [1/10], Step [30/39], Loss: 0.5282
Epoch [2/10], Step [10/39], Loss: 0.4948
Epoch [2/10], Step [20/39], Loss: 0.4731
Epoch [2/10], Step [30/39], Loss: 0.4765
Epoch [3/10], Step [10/39], Loss: 0.3961
Epoch [3/10], Step [20/39], Loss: 0.4059
Epoch [3/10], Step [30/39], Loss: 0.4316
Epoch [4/10], Step [10/39], Loss: 0.4236
Epoch [4/10], Step [20/39], Loss: 0.4024
Epoch [4/10], Step [30/39], Loss: 0.3569
Epoch [5/10], Step [10/39], Loss: 0.3071
Epoch [5/10], Step [20/39], Loss: 0.3190
Epoch [5/10], Step [30/39], Loss: 0.3566
Epoch [6/10], Step [10/39], Loss: 0.3329
Epoch [6/10], Step [20/39], Loss: 0.3434
Epoch [6/10], Step [30/39], Loss: 0.3486
Epoch [7/10], Step [10/39], Loss: 0.3564
Epoch [7/10], Step [20/39], Loss: 0.4210
Epoch [7/10], Step [30/39], Loss: 0.3429
Epoch [8/10], Step [10/39], Loss: 0.4517
Epoch [8/10], Step [20/39], Loss: 0.3986
Epoch [8/10], Step [30/39], Loss: 0.3586
Epoch [9/10], St

In [53]:
# Probability Threshold for M6A modification
threshold = 0.8

# Test the model: we don't need to compute gradients
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    
    for features, labels in test_dataLoader:
        # Flatten labels
        labels = labels.flatten().float()
        
        # Obtain prediction probabilities
        y_predict = model(features)
        # Convert to classification based on threshold
        y_predict = (y_predict>threshold).float()
        
        # No. of correct predictions
        n_correct += (y_predict==labels).sum().item()
        n_samples += len(labels)
        
    acc = n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images: {100*acc} %')

Accuracy of the network on the 171 test images: 91.81286549707602 %


In [146]:
# Probability Threshold for M6A modification
threshold = 0.8

# Store prediction scores
results_scores = torch.tensor([])
# Store Prediction Classes
results_class = torch.tensor([])

# Test the model: we don't need to compute gradients
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    
    for features, labels in test_dataLoader:
        # Flatten labels
        labels = labels.flatten().float()
        
        # Obtain prediction probabilities
        y_predict = model(features)
        # Store Predicted probabilities
        results_scores = torch.concat([results_scores, y_predict])
        # Convert to classification based on threshold
        y_predict = (y_predict>threshold).float()
        # Store Predicted classes
        results_class = torch.concat([results_class, y_predict])
        
        # No. of correct predictions
        n_correct += (y_predict==labels).sum().item()
        n_samples += len(labels)
        
    acc = n_correct / n_samples
    print(f'Accuracy of the network on the {n_samples} test images: {100*acc} %')

Accuracy of the network on the 171 test images: 94.73684210526315 %


In [134]:
q = testData.df[['gene_id', 'transcript', 'position', 'label']].copy()
q

,gene_id,transcript,position,label
2393,ENSG00000240972,ENST00000215754,490,0
2394,ENSG00000240972,ENST00000215754,604,0
2395,ENSG00000240972,ENST00000215754,688,0
2396,ENSG00000240972,ENST00000215754,748,0
2397,ENSG00000240972,ENST00000215754,799,0
...,...,...,...,...
9277,ENSG00000260027,ENST00000239165,921,0
9278,ENSG00000260027,ENST00000239165,1147,1
9279,ENSG00000260027,ENST00000239165,1160,0
9280,ENSG00000260027,ENST00000239165,1164,0


In [130]:
q['pred_class'] = np.array(results_class)
q['pred_class'] = q['pred_class'].astype(int)

In [131]:
q

,gene_id,transcript,position,k-mer bases,values,label,pred_class
2393,ENSG00000240972,ENST00000215754,490,TAAACAC,"[[0.0131, 2.27, 103.0, 0.00498, 2.93, 97.6, 0....",0,0
2394,ENSG00000240972,ENST00000215754,604,CGGACCA,"[[0.00631, 4.64, 113.0, 0.00963, 10.7, 120.0, ...",0,0
2395,ENSG00000240972,ENST00000215754,688,AGAACCG,"[[0.0102, 6.85, 131.0, 0.00638, 3.74, 95.9, 0....",0,0
2396,ENSG00000240972,ENST00000215754,748,CGGACAG,"[[0.00498, 2.45, 119.0, 0.00332, 5.41, 120.0, ...",0,0
2397,ENSG00000240972,ENST00000215754,799,GGAACAA,"[[0.0083, 9.35, 123.0, 0.00799, 2.17, 95.2, 0....",0,0
...,...,...,...,...,...,...,...
9277,ENSG00000260027,ENST00000239165,921,AAAACAT,"[[0.00256, 3.52, 105.0, 0.00835, 3.85, 100.0, ...",0,0
9278,ENSG00000260027,ENST00000239165,1147,TGGACTG,"[[0.00879, 3.71, 115.0, 0.00564, 5.64, 117.0, ...",1,1
9279,ENSG00000260027,ENST00000239165,1160,TGGACTA,"[[0.0105, 2.97, 120.0, 0.00855, 10.4, 116.0, 0...",0,0
9280,ENSG00000260027,ENST00000239165,1164,CTAACCC,"[[0.00518, 2.28, 92.3, 0.0155, 3.89, 96.8, 0.0...",0,0


In [150]:
np.argmax(np.array(results_class))

18

In [136]:
label_df.shape

(121838, 4)

In [137]:
round(0.7*121838)

85287

In [139]:
85287/128

666.3046875

In [106]:
results_class.shape

torch.Size([171])

In [105]:
q = np.array(testData.df['label'].to_list())

torch.mean((torch.from_numpy(q)==results_class).float())

tensor(0.9357)

In [54]:
# Save model parameters

path = "./m6a.pth"
torch.save(model.state_dict(), path)

In [65]:
# Load model paramaters
model = m6aNet(batchsize=batchsize, readsize=20)
model.load_state_dict(torch.load(path)) 
# model.eval()

<All keys matched successfully>

In [66]:
model.eval()

m6aNet(
  (embed): Embedding(66, 2)
  (read_level_prob_1): Linear(in_features=15, out_features=150, bias=True)
  (norm_1): BatchNorm1d(150, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activ_1): ReLU()
  (drop_1): Dropout(p=0.0, inplace=False)
  (read_level_prob_2): Linear(in_features=150, out_features=32, bias=True)
  (activ_2): ReLU()
  (drop_2): Dropout(p=0.0, inplace=False)
  (read_level_prob_3): Linear(in_features=32, out_features=1, bias=True)
  (sig_1): Sigmoid()
)

In [91]:
a = torch.tensor([1,2,3])
b = torch.tensor([4, 5])

In [82]:
c = torch.concat([a, b])

In [85]:
y_predict.shape

torch.Size([171])

In [86]:
a[1]=100

In [87]:
c

tensor([1, 2, 3, 4, 5])

In [88]:
q = torch.tensor([])

In [92]:
torch.concat((q, a, b))

tensor([1., 2., 3., 4., 5.])

In [109]:
%run -i "../scripts/run_learner.py"

This is a test
['/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/notebooks', '/Users/carelchay/opt/anaconda3/envs/DS/lib/python39.zip', '/Users/carelchay/opt/anaconda3/envs/DS/lib/python3.9', '/Users/carelchay/opt/anaconda3/envs/DS/lib/python3.9/lib-dynload', '', '/Users/carelchay/opt/anaconda3/envs/DS/lib/python3.9/site-packages', '/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/scripts']
/Users/carelchay/Desktop/School/Modules/DSA4262/Project 2/DSA4262-frontasticfour/notebooks


In [110]:
from m6aNet import m6aNet

In [113]:
import math

In [115]:
type(math.ceil(123/2))

int

In [116]:
import numpy as np

In [118]:
q = np.array([])

In [119]:
np.concat([])

array([], dtype=float64)

In [132]:
q

,gene_id,transcript,position,k-mer bases,values,label,pred_class
2393,ENSG00000240972,ENST00000215754,490,TAAACAC,"[[0.0131, 2.27, 103.0, 0.00498, 2.93, 97.6, 0....",0,0
2394,ENSG00000240972,ENST00000215754,604,CGGACCA,"[[0.00631, 4.64, 113.0, 0.00963, 10.7, 120.0, ...",0,0
2395,ENSG00000240972,ENST00000215754,688,AGAACCG,"[[0.0102, 6.85, 131.0, 0.00638, 3.74, 95.9, 0....",0,0
2396,ENSG00000240972,ENST00000215754,748,CGGACAG,"[[0.00498, 2.45, 119.0, 0.00332, 5.41, 120.0, ...",0,0
2397,ENSG00000240972,ENST00000215754,799,GGAACAA,"[[0.0083, 9.35, 123.0, 0.00799, 2.17, 95.2, 0....",0,0
...,...,...,...,...,...,...,...
9277,ENSG00000260027,ENST00000239165,921,AAAACAT,"[[0.00256, 3.52, 105.0, 0.00835, 3.85, 100.0, ...",0,0
9278,ENSG00000260027,ENST00000239165,1147,TGGACTG,"[[0.00879, 3.71, 115.0, 0.00564, 5.64, 117.0, ...",1,1
9279,ENSG00000260027,ENST00000239165,1160,TGGACTA,"[[0.0105, 2.97, 120.0, 0.00855, 10.4, 116.0, 0...",0,0
9280,ENSG00000260027,ENST00000239165,1164,CTAACCC,"[[0.00518, 2.28, 92.3, 0.0155, 3.89, 96.8, 0.0...",0,0


In [135]:
q

,gene_id,transcript,position,label
2393,ENSG00000240972,ENST00000215754,490,0
2394,ENSG00000240972,ENST00000215754,604,0
2395,ENSG00000240972,ENST00000215754,688,0
2396,ENSG00000240972,ENST00000215754,748,0
2397,ENSG00000240972,ENST00000215754,799,0
...,...,...,...,...
9277,ENSG00000260027,ENST00000239165,921,0
9278,ENSG00000260027,ENST00000239165,1147,1
9279,ENSG00000260027,ENST00000239165,1160,0
9280,ENSG00000260027,ENST00000239165,1164,0


In [ ]:
pd.DataFrame({
    "epoch_num" = [i for i in range(1, 10+1)],
    
    
})